<a href="https://colab.research.google.com/github/manujism/Risk-Modelling/blob/main/Risk-Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
# Importing Libraries and Data Collection

import numpy as np
import yfinance as yf
import pandas as pd
from scipy.stats import norm

portfolio = ['MSFT', 'AAPL', 'NVDA', 'META', 'TSLA']
data = yf.download(portfolio, start='2020-01-01', end='2025-06-11', progress=False)['Close']
returns = data.pct_change().dropna()

data.head()

Ticker,AAPL,META,MSFT,NVDA,TSLA
Date,,,,,
2020-01-02,72.620834,208.795929,153.042282,5.972162,28.684000
2020-01-03,71.914803,207.691132,151.136673,5.876571,29.534000
2020-01-06,72.487846,211.602707,151.527313,5.901215,30.102667
2020-01-07,72.146942,212.060562,150.145737,5.972661,31.270666
2020-01-08,73.307518,214.210434,152.537262,5.983862,32.809334


In [85]:
# Portfolio Settings

weights = [0.20, 0.20, 0.20, 0.20, 0.20]   # Equal weighting
confidence = 0.95
Portfolio_Value = 100
portfolio_returns = returns.dot(weights) * Portfolio_Value

portfolio_returns.head()

,0
Date,
2020-01-03,-0.276756
2020-01-06,1.056696
2020-01-07,0.785017
2020-01-08,1.864650
2020-01-09,0.741644


In [86]:
# Parametric Method

std = portfolio_returns.std()
neu = portfolio_returns.mean()
z_score = norm.ppf(confidence)
std_norm_pdf = norm.pdf(z_score)

VaR_P = - (neu - std * z_score)
CvaR_P = - (neu - (std_norm_pdf / (1 - confidence)) * std)



In [87]:
# Historical Method

pnl = np.sort(portfolio_returns)
percentile_index = int((1 - confidence) * len(portfolio_returns))

VaR_H = -pnl[percentile_index]
CVaR_H = -pnl[:percentile_index].mean()


In [88]:
# Monte Carlo Simulation

np.random.seed(123)

simulated_returns = np.random.normal(
    loc=portfolio_returns.mean(),
    scale=portfolio_returns.std(),
    size=100000
)
sorted_pnl_simulated = np.sort(simulated_returns)
percentile_index_sorted = int((1 - confidence) * len(sorted_pnl_simulated))

VaR_MC = -sorted_pnl_simulated[percentile_index_sorted]
CVaR_MC = -sorted_pnl_simulated[:percentile_index_sorted].mean()


print(portfolio_returns_simulated)

[-233.23452468  248.38511472   83.21127503 ...  285.72382006  545.82280733
 -287.02507679]


In [89]:
# Printing The Results

print(f"Parametric VaR: ${VaR_P:.2f}")
print(f"Parametric CVaR: ${CvaR_P:.2f}")
print(f"Historical VaR: ${VaR_H:.2f}")
print(f"Historical CVaR: ${CVaR_H:.2f}")
print(f"Monte Carlo VaR: ${VaR_MC:.2f}")
print(f"Monte Carlo CVaR: ${CVaR_MC:.2f}")

Parametric VaR: $3.63
Parametric CVaR: $4.59
Historical VaR: $3.64
Historical CVaR: $5.23
Monte Carlo VaR: $3.61
Monte Carlo CVaR: $4.57
